In [31]:
import os
import numpy as np
import cv2
import matplotlib.pylab as plt
from sklearn.cluster import KMeans
import json
import random

**PATHS**

In [32]:
image_path      = '../../../dataset/images'
training_path   = os.path.join(image_path, 'training')
original_path   = os.path.join(training_path, 'original')
processed_path  = os.path.join(training_path, 'processed')

**LISTAS DE IMAGENES**

In [33]:
original  = [os.path.join(original_path, image) for image in os.listdir(original_path)]
processed  = [os.path.join(processed_path, image) for image in os.listdir(processed_path)]

**RANGOS DE COLOR**

In [34]:
lower_green = np.array([28, 40, 40])
upper_green = np.array([100, 255, 255])

lower_red_1 = np.array([0, 120, 100])
upper_red_1 = np.array([5, 255, 255])

lower_red_2 = np.array([170, 120, 100])
upper_red_2 = np.array([179, 255, 255])

lower_yellow = np.array([18, 50, 80])
upper_yellow = np.array([33, 255, 255])

lower_orange = np.array([5, 120, 90])
upper_orange = np.array([20, 255, 255])

**PROCESAMIENTO DE LAS IMAGENES**
- Primera prueba

In [ ]:
for k, file in enumerate(original):
    # BGR image
    image = cv2.imread(file)

    # Dimentions
    height, width, _ = image.shape
 
    # Pixel data vector
    data_vector = np.zeros((height * width, 4))

    # Obtener matrices de color
    rgb_matrix = image.reshape((-1, 3))
    hsv_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).reshape((-1, 3))
    lab_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2LAB).reshape((-1, 3))

    # Asignar a la matriz de datos
    data_vector[:, 0]  = rgb_matrix[:, 2]
    data_vector[:, 1]  = hsv_matrix[:, 1]
    data_vector[:, 2:] = lab_matrix[:, 1:]

    kmeans = KMeans(n_clusters = 2, n_init = 10)  # 2 Clusters. Background and fruit
    kmeans.fit(data_vector)

    # Get clusters labels
    labels = kmeans.labels_

    # kmeans_mask
    kmeans_mask = labels.reshape(height, width)
    kmeans_mask = kmeans_mask.astype(np.uint8) * 255

    # Determinación del tipo de fondo de la máscara

    # La estrategia es análizar una matriz de pixeles en todas las esquinas
    # de un orden cuadrado y también en los bordes de la mágen y contar la 
    # proporción del total que es blanca y la proporción que es negra.

    # Se define el tmaño del cluster relativo a las dimensiones de la imàge
    # dado que distintas imàgenes tienen distintos tamaños
    cluster_size = min([height, width])//20
    cluster      = np.ones((cluster_size, cluster_size), np.uint8)

    # Corners
    corner1 = np.bitwise_and(cluster, kmeans_mask[:cluster_size,  :cluster_size])
    corner2 = np.bitwise_and(cluster, kmeans_mask[:cluster_size:, -cluster_size:])
    corner3 = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, :cluster_size])
    corner4 = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, -cluster_size:])
    corners = [corner1, corner2, corner3, corner4]

    # Sides
    limitw1 = (width - cluster_size)//2
    limitw2 = (width + cluster_size)//2
    limith1 = (height - cluster_size)//2
    limith2 = (height + cluster_size)//2
    
    side1   = np.bitwise_and(cluster, kmeans_mask[:cluster_size, limitw1:limitw2])
    side2   = np.bitwise_and(cluster, kmeans_mask[limith1:limith2, :cluster_size])
    side3   = np.bitwise_and(cluster, kmeans_mask[limith1:limith2, -cluster_size:])
    side4   = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, limitw1:limitw2])
    sides   = [side1, side2, side3, side4] 

    
    # Determining the type of background
    edges            = corners + sides
    light_background = sum(np.count_nonzero(edge) for edge in edges) > 0.75*8*(cluster_size**2)

    # Inverting if dark background
    if light_background:
        print(f"blanco-{os.path.basename(file)}")
        kmeans_mask = np.bitwise_not(kmeans_mask)

    # Encontrar contornos
    kmeans_cnt, _ = cv2.findContours(kmeans_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tkmeans = np.zeros((height, width), dtype=np.uint8)

    # Dibujar
    cv2.drawContours(tkmeans, kmeans_cnt, -1, 255, thickness = cv2.FILLED)

    # Imagen de salida
    image_out = cv2.bitwise_and(image, image, mask = tkmeans)
    cv2.imwrite(os.path.join(processed_path, os.path.basename(file)), image_out)

- Seguinda prueba ( Otra vez con el algoritmo genètico). NO funciono bien.

In [42]:
def get_extenssion(mask, contorno, num_cuadriculas_x, num_cuadriculas_y):
    h, w = mask.shape
    # Calcular el ancho y la altura de cada cuadrícula
    ancho_cuadricula = w // num_cuadriculas_x
    altura_cuadricula = h // num_cuadriculas_y

    # Inicializar el contador de cuadrículas atravesadas por el contorno
    cuadriculas_atravesadas = 0
    cuadriculas_contadas = set()

    # Verificar por cuántas cuadrículas pasa el contorno
    for punto in contorno[:, 0, :]:
        # Obtener las coordenadas (x, y) del punto
        x, y = punto

        # Determinar la cuadrícula a la que pertenece el punto
        cuadricula_x = x // ancho_cuadricula
        cuadricula_y = y // altura_cuadricula

        # Aumentar el contador si la cuadrícula no ha sido contada previamente
        if (cuadricula_x, cuadricula_y) not in cuadriculas_contadas:
            cuadriculas_atravesadas += 1
            cuadriculas_contadas.add((cuadricula_x, cuadricula_y))

    return cuadriculas_atravesadas

In [43]:
def get_light_background(mask, f = 20):
    height, width = mask.shape
    cluster_size  = min([height, width])//f
    cluster       = np.ones((cluster_size, cluster_size), np.uint8)

    # Corners
    corner1 = np.bitwise_and(cluster, mask[:cluster_size,  :cluster_size])
    corner2 = np.bitwise_and(cluster, mask[:cluster_size:, -cluster_size:])
    corner3 = np.bitwise_and(cluster, mask[-cluster_size:, :cluster_size])
    corner4 = np.bitwise_and(cluster, mask[-cluster_size:, -cluster_size:])
    corners = [corner1, corner2, corner3, corner4]

    # Sides
    limitw1 = (width - cluster_size)//2
    limitw2 = (width + cluster_size)//2
    limith1 = (height - cluster_size)//2
    limith2 = (height + cluster_size)//2
    
    side1   = np.bitwise_and(cluster, mask[:cluster_size, limitw1:limitw2])
    side2   = np.bitwise_and(cluster, mask[limith1:limith2, :cluster_size])
    side3   = np.bitwise_and(cluster, mask[limith1:limith2, -cluster_size:])
    side4   = np.bitwise_and(cluster, mask[-cluster_size:, limitw1:limitw2])
    sides   = [side1, side2, side3, side4] 
    
    # Determining the type of background
    edges            = corners + sides
    light_background = sum(np.count_nonzero(edge) for edge in edges) > 0.75*8*(cluster_size**2)

    # Inverting if dark background
    if light_background:
        return np.bitwise_not(mask)
    return mask    

In [58]:
def genetic_algorithm(masks, max_pob = 100, p_mutacion = 0.001, p_cruce = 0.98, n_iter = 3):
    # Econtrar contornos
    final_masks   = []
    final_masks_idoneidad = []
    for _ in range(n_iter):
        masks_cntrs   = [cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0] for mask in masks]
        heigth, width = masks[0].shape
        pool          = []
        resto         = len(masks)%2
        for p in range(max_pob):

            # Numero de contornos por mascara
            nro_cntrs  = [len(cntrs) for cntrs in masks_cntrs]

            # Contorno mas largo por mascara
            lrgst_cntrs = [max(cntrs, key=cv2.contourArea) for cntrs in masks_cntrs]

            # Obtención del ratio promedio de los contornos por cada mascara
            cntrs_ratios = []
            for cntrs in masks_cntrs:
                ratios = []
                for cntr in cntrs:
                    length = cv2.arcLength(cntr, closed=False)
                    if length <= 0.0:
                        ratios.append(length)
                    else:
                        ratios.append(cv2.contourArea(cntr)/length)
                cntrs_ratios.append(np.mean(np.array(ratios)))

            # Determinar si el contorno mas largo de cada mascara es cerrado o no
            lrgst_cntr_closed = [cv2.isContourConvex(cntr) for cntr in lrgst_cntrs]

            # Determinar la extensión de los contornos
            # Podemos determinar la calidad de la extensión de los contornos
            # Como la extensión promedio. Es decir, la suma de las extensiones de los contornos dividido el numero de ellos

            # Extensiones de los contornos mas largos
            lrgst_cntr_extenssion = [get_extenssion(mask, lrgst_cntrs[i], 4, 4)/16 for i, mask in enumerate(masks)]

            # Extensión promedio de los contornos en la mascara
            extenssions = [np.mean(np.array([get_extenssion(mask, contour, 4, 4)/16 for contour in masks_cntrs[i]])) for i, mask in enumerate(masks)]

            # Ahora obtener la función de idoneidad para cada mascara
            # Por el momento no vamos a utilizar las longitudes acumuladas sino mas bien los ratios promedios de los contornos de las mascaras
            total_cntrs            = sum(nro_cntrs)
            total_ratio            = sum(cntrs_ratios)
            total_extenssion       = sum(extenssions)
            total_lrgst_extenssion = sum(lrgst_cntr_extenssion)

            #------------------------------------------------Càlculo de idoneidad-----------------------------------------------------

            idoneidad = [(total_cntrs - nro)/((len(nro_cntrs) - 1)*total_cntrs) for nro in nro_cntrs]
            if total_ratio > 0:
                aux = [idoneidad[i] + ratio/total_ratio for i, ratio in enumerate(cntrs_ratios)] 
                idoneidad = [value/sum(aux) for value in aux]
            if total_extenssion > 0:
                aux = [idoneidad[i] + extenssion/total_extenssion for i, extenssion in enumerate(extenssions)]
                idoneidad = [value/sum(aux) for value in aux]
            aux = [idoneidad[i]*1.07 if closed else idoneidad[i] for i, closed in enumerate(lrgst_cntr_closed)]
            idoneidad = [value/sum(aux) for value in aux]
            if total_lrgst_extenssion > 0:
                aux = [idoneidad[i] + lrgst_ext/total_lrgst_extenssion for i, lrgst_ext in enumerate(lrgst_cntr_extenssion)]
                idoneidad = [value/sum(aux) for value in aux]
            aux = [idoneidad[i]*1.2 if nro == 1 else idoneidad[i] for i, nro in enumerate(nro_cntrs)]
            idoneidad = [value/sum(aux) for value in aux]

            # Break cuando no hay mas variacion
            if all(nro == 1 for nro in nro_cntrs):
                if all(cntrs[0].shape == masks_cntrs[0][0].shape for cntrs in masks_cntrs):
                    if all((cntrs[0] == masks_cntrs[0][0]).all for cntrs in masks_cntrs):
                        break

            # ------------------------------------------------Calculo de poblaciòn----------------------------------------------------
            poblacion = []
            for _ in range(len(masks)//2 + resto):
                index = []
                # ----------------------------------Seleccion-----------------------------------------------
                for _ in range(2):
                    aguja = random.random()
                    total = 0
                    for j, value in enumerate(idoneidad):
                        total += value
                        if aguja < total:
                            break
                    index.append(j)
                index = tuple(index)

                hijos = [] # Las listas de contornos de los hijos

                # ---------------------------------------cruce--------------------------------------------
                if random.random() < p_cruce:
                    pases = []   # Las listas de contornos que se pasan
                    for ind in index:
                        hijo = list(masks_cntrs[ind])
                        pase = []
                        for _ in range(nro_cntrs[ind]//2):
                            k = random.choice(list(range(len(hijo))))
                            cntr = hijo.pop(k)
                            pase.append(cntr)
                        pases.append(pase)
                        hijos.append(hijo)
                    hijos[0] = hijos[0] + pases[1]
                    hijos[1] = hijos[1] + pases[0]
                else:
                    hijos.append(list(masks_cntrs[index[0]]))
                    hijos.append(list(masks_cntrs[index[1]]))

                #---------------------------------------mutación-------------------------------------------
                if random.random() < p_mutacion:
                    for j, hijo in enumerate(hijos):
                        total_area = sum([cv2.contourArea(cntr) for cntr in hijo])
                        for i, cntr in enumerate(hijo):
                            if total_area > 0:
                                ratio = cv2.contourArea(cntr)/total_area
                            else:
                                ratio = 0
                            if random.random() > ratio:
                                pool.append(hijos[j].pop(i))
                            if random.choice([True, True]):
                                if pool:
                                    hijos[j].append(pool.pop(random.randint(0, len(pool)-1)))
                #----------------------------------------Poblacion-----------------------------------------
                poblacion = poblacion + hijos
            # En caso de un nùmero impar de mascaras sobra un elemento
            if resto != 0:
                x = random.randint(0, len(poblacion) - 1)
                poblacion.pop(x)
            
            # Pasamos a tuplas para la funciòn de drawContours
            for j, cntrs in enumerate(poblacion):
                poblacion[j] = tuple(cntrs)

            #-------------------------------------------Poblaciòn final-------------------------------------

            masks       = []
            masks_cntrs = []

            #-------------------------------------------#Acutalizar poblacion-------------------------------
            for cntrs in poblacion:
                mask = np.zeros((height, width), dtype = np.uint8)
                cv2.drawContours(mask, cntrs, -1, 255, 1)
                masks.append(mask)
                cntrs, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                masks_cntrs.append(cntrs)
        # Devolvemos la màscara rellena
        mask      = masks[0]
        cntrs     = masks_cntrs[0]
        idn       = idoneidad[0]
        cv2.drawContours(mask, cntrs, -1, (255), thickness = cv2.FILLED)
        final_masks.append(mask)
        final_masks_idoneidad.append(idn)
    indice = final_masks_idoneidad.index(max(final_masks_idoneidad))
    return final_masks[indice]

In [59]:
for k, file in enumerate(original):
    # BGR image
    image = cv2.imread(file)

    # Dimentions
    height, width, _ = image.shape
 
    # Pixel data vector
    data_vector = np.zeros((height * width, 4))

    # Otras mascaras
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, s, _   = cv2.split(hsv_image)
    _, _, b   = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2LAB))
    _, s_mask = cv2.threshold(s, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, b_mask = cv2.threshold(b, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    mask_red    = cv2.inRange(hsv_image, lower_red_1, upper_red_1) | cv2.inRange(hsv_image, lower_red_2, upper_red_2)
    mask_orange = cv2.inRange(hsv_image, lower_orange, upper_orange)
    mask_green  = cv2.inRange(hsv_image, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

    # Unir las máscaras
    combined_mask   = mask_red | mask_orange | mask_green | mask_yellow

    # Obtener matrices de color
    rgb_matrix = image.reshape((-1, 3))
    hsv_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).reshape((-1, 3))
    lab_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2LAB).reshape((-1, 3))

    # Asignar a la matriz de datos
    data_vector[:, 0]  = rgb_matrix[:, 2]
    data_vector[:, 1]  = hsv_matrix[:, 1]
    data_vector[:, 2:] = lab_matrix[:, 1:]

    kmeans = KMeans(n_clusters = 2, n_init = 10)  # 2 Clusters. Background and fruit
    kmeans.fit(data_vector)

    # Get clusters labels
    labels = kmeans.labels_

    # kmeans_mask
    kmeans_mask = labels.reshape(height, width)
    kmeans_mask = kmeans_mask.astype(np.uint8) * 255

    # Llevar las màscaras a fondo blanco: s, b, kmean
    s_mask      = get_light_background(b_mask)
    b_mask      = get_light_background(s_mask)
    kmeans_mask = get_light_background(kmeans_mask)
    masks       = [s_mask, b_mask, kmeans_mask]
    mask        = genetic_algorithm(masks)
    '''# Templates
    tkmeans = np.zeros((height, width), dtype=np.uint8)
    ts      = np.zeros((height, width), dtype=np.uint8)
    tb      = np.zeros((height, width), dtype=np.uint8)

    # Dibujar
    cv2.drawContours(tkmeans, kmeans_cnt, -1, 255, 1)
    cv2.drawContours(ts, s_cnt, -1, 255, 1)
    cv2.drawContours(tb, b_cnt, -1, 255, 1)'''

    # Imagen de salida
    image_out = cv2.bitwise_and(image, image, mask = mask)
    cv2.imwrite(os.path.join(processed_path, os.path.basename(file)), image_out)

Lo ùnico que queda por hacer es una erosiòn y dilataciòn de la màscara una vez que se encuentra la mascara de kmeans

In [79]:
files = [os.path.join(original_path, 'fruit99.jpg')]
for k, file in enumerate(files):
    # BGR image
    image = cv2.imread(file)

    # Dimentions
    height, width, _ = image.shape
 
    # Pixel data vector
    data_vector = np.zeros((height * width, 4))

    # Obtener matrices de color
    rgb_matrix = image.reshape((-1, 3))
    hsv_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).reshape((-1, 3))
    lab_matrix = cv2.cvtColor(image, cv2.COLOR_BGR2LAB).reshape((-1, 3))

    # Asignar a la matriz de datos
    data_vector[:, 0]  = rgb_matrix[:, 2]
    data_vector[:, 1]  = hsv_matrix[:, 1]
    data_vector[:, 2:] = lab_matrix[:, 1:]

    kmeans = KMeans(n_clusters = 2, n_init = 10)  # 2 Clusters. Background and fruit
    kmeans.fit(data_vector)

    # Get clusters labels
    labels = kmeans.labels_

    # kmeans_mask
    kmeans_mask = labels.reshape(height, width)
    kmeans_mask = kmeans_mask.astype(np.uint8) * 255

    # Determinación del tipo de fondo de la máscara

    # La estrategia es análizar una matriz de pixeles en todas las esquinas
    # de un orden cuadrado y también en los bordes de la mágen y contar la 
    # proporción del total que es blanca y la proporción que es negra.

    # Se define el tmaño del cluster relativo a las dimensiones de la imàge
    # dado que distintas imàgenes tienen distintos tamaños
    cluster_size = min([height, width])//20
    cluster      = np.ones((cluster_size, cluster_size), np.uint8)

    # Corners
    corner1 = np.bitwise_and(cluster, kmeans_mask[:cluster_size,  :cluster_size])
    corner2 = np.bitwise_and(cluster, kmeans_mask[:cluster_size:, -cluster_size:])
    corner3 = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, :cluster_size])
    corner4 = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, -cluster_size:])
    corners = [corner1, corner2, corner3, corner4]

    # Sides
    limitw1 = (width - cluster_size)//2
    limitw2 = (width + cluster_size)//2
    limith1 = (height - cluster_size)//2
    limith2 = (height + cluster_size)//2
    
    side1   = np.bitwise_and(cluster, kmeans_mask[:cluster_size, limitw1:limitw2])
    side2   = np.bitwise_and(cluster, kmeans_mask[limith1:limith2, :cluster_size])
    side3   = np.bitwise_and(cluster, kmeans_mask[limith1:limith2, -cluster_size:])
    side4   = np.bitwise_and(cluster, kmeans_mask[-cluster_size:, limitw1:limitw2])
    sides   = [side1, side2, side3, side4] 

    
    # Determining the type of background
    edges            = corners + sides
    light_background = sum(np.count_nonzero(edge) for edge in edges) > 0.75*8*(cluster_size**2)

    # Inverting if dark background
    if light_background:
        print(f"blanco-{os.path.basename(file)}")
        kmeans_mask = np.bitwise_not(kmeans_mask)

    erosion_size      = min([height, width])//200
    dilatacion_size   = min([height, width])//80
    kernel_erosion    = np.ones((erosion_size,erosion_size), np.uint8)
    eroded            = cv2.erode(kmeans_mask, kernel_erosion, iterations = 1)
    kernel_dilatacion = np.ones((dilatacion_size,dilatacion_size), np.uint8)
    kmeans_mask       = cv2.dilate(eroded, kernel_dilatacion, iterations  = 2)

    # Encontrar contornos
    kmeans_cnt, _ = cv2.findContours(kmeans_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tkmeans = np.zeros((height, width), dtype=np.uint8)

    # Dibujar
    cv2.drawContours(tkmeans, kmeans_cnt, -1, 255, thickness = cv2.FILLED)

    # Imagen de salida
    image_out = cv2.bitwise_and(image, image, mask = tkmeans)
    cv2.imwrite(os.path.join(processed_path, os.path.basename(file)), image_out)